# Labelエンコーディング

カテゴリ変数をそのまま数値に対応付けるもの．  
決定木系などは値を場合分けして分岐するのでうまく機能するが，線形モデルなどw*xのような形になっているものに対してはそのまま演算に利用されるのでうまく機能しないことに注意．

# One-Hotエンコーディング

One-Hotエンコーディングは，すべて0の組み合わせを作らない．よって自由度がkとなる．

これはエンコーディングしたカテゴリ変数の和が1となる，つまり特徴量に線形の依存関係があることを指し，線形モデルの係数が一意に定まらないという問題を生じる．その結果，特徴量が予測に与える影響を理解するのが難しくなる．

In [1]:
import pandas as pd
from sklearn import linear_model

In [2]:
df = pd.DataFrame({
    'City': ['SF', 'SF', 'SF', 'NYC', 'NYC', 'NYC', 'Seattle', 'Seattle', 'Seattle'],
    'Rent': [3999, 4000, 4001, 3499, 3500, 3501, 2499, 2500, 2501]
})

df['Rent'].mean()

3333.3333333333335

In [3]:
one_hot_df = pd.get_dummies(df, prefix=['city'])
one_hot_df

,Rent,city_NYC,city_SF,city_Seattle
0,3999,0,1,0
1,4000,0,1,0
2,4001,0,1,0
3,3499,1,0,0
4,3500,1,0,0
5,3501,1,0,0
6,2499,0,0,1
7,2500,0,0,1
8,2501,0,0,1


In [4]:
model = linear_model.LinearRegression()
model.fit(one_hot_df[['city_NYC', 'city_SF', 'city_Seattle']], one_hot_df['Rent'])
model.coef_

array([ 166.66666667,  666.66666667, -833.33333333])

In [5]:
model.intercept_

3333.3333333333335

One-Hotエンコーディングでは，切片はターゲット変数の平均値（全体の平均賃料）になり，各特徴量の回帰係数は，各カテゴリの平均賃料と全体平均の差分となる．

# ダミーコーディング

ダミーコーディングはすべて0の組み合わせをある１つのカテゴリに割り当てる（参照カテゴリ）ので，自由度がk-1となる．  


In [6]:
dummy_df = pd.get_dummies(df, prefix=['city'], drop_first=True)
dummy_df

,Rent,city_SF,city_Seattle
0,3999,1,0
1,4000,1,0
2,4001,1,0
3,3499,0,0
4,3500,0,0
5,3501,0,0
6,2499,0,1
7,2500,0,1
8,2501,0,1


NYCが参照カテゴリとして扱われ，SFとSeattleが双方0の場合となる．

In [7]:
model.fit(dummy_df[['city_SF', 'city_Seattle']], dummy_df['Rent'])
model.coef_

array([  500., -1000.])

city_NYCの係数=0が隠れている

In [8]:
model.intercept_

3500.0

ダミーエンコーディングでは，切片は参照カテゴリのターゲット変数の平均値，各特徴量の回帰係数は，各カテゴリターゲット変数の平均値と参照カテゴリの平均値との差

# Effectエンコーディング

ダミーエンコーディングの参照カテゴリが-1になる．

In [11]:
effect_df = dummy_df.copy()
effect_df.loc[3:5, ['city_SF', 'city_Seattle']] = -1.0
effect_df

,Rent,city_SF,city_Seattle
0,3999,1.0,0.0
1,4000,1.0,0.0
2,4001,1.0,0.0
3,3499,-1.0,-1.0
4,3500,-1.0,-1.0
5,3501,-1.0,-1.0
6,2499,0.0,1.0
7,2500,0.0,1.0
8,2501,0.0,1.0


In [12]:
model.fit(effect_df[['city_SF', 'city_Seattle']], effect_df['Rent'])
model.coef_

array([ 666.66666667, -833.33333333])

In [13]:
model.intercept_

3333.3333333333335

切片と各カテゴリの係数の考え方は，One-Hotエンコーディングと同じ  
各カテゴリの，全体平均との差分を「各カテゴリの主効果（main effect）」と呼ぶ．  
参照カテゴリの係数は，参照カテゴリ以外のカテゴリの和をとり正負反対にする

# 使い分け

- One-Hotエンコーディングは冗長な表現であるため，同じ問題に対して妥当なモデルが複数存在し係数が一意に定まらない．よって，結果の解釈の妨げにつながることがある．しかし，カテゴリが欠損しているデータに対しすべて０のレコードを作れるので欠損データを扱うときは便利．
- ダミーエンコーディングおよびEffectエンコーディングはユニークかつ解釈可能なモデルを得られる．しかし参照カテゴリにより欠損データを簡単に扱えない．また，参照カテゴリに対する各カテゴリの相対的な影響をエンコードすることは少し不自然．
- Effectエンコーディングはこの問題を解決するが参照カテゴリはすべて-1の密ベクトルになるため，記憶容量や計算コストがかさむ．よって，Pandasやscikit-learnのようなソフトウェアパッケージは上２つが使われている．

# 特徴量ハッシング(ハッシングトリック)

大規模なカテゴリ変数を扱うときに使われる（インターネットニュース記事など）．  
潜在的に無限に広がる値を有限のm種類の値に割り当てるハッシュ関数をあてる．同じハッシュに割り当てる異なる入力は衝突と呼ばれる．

線形モデルにおいてよく使われる

In [16]:
import pandas as pd
import json

# 店舗データ
with open('data/Yelp/review.json', encoding='utf-8') as review_file:
    js = []
    for i in range(10000):
        js.append(json.loads(review_file.readline()))
                  
reveiw_df = pd.DataFrame(js)

In [17]:
reveiw_df.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,ujmEBvifdJM6h6RLv4wQIg,0,2013-05-07 04:34:36,1,Q1sbwvVQXV2734tPgoKj4Q,1.0,Total bill for this horrible service? Over $8G...,6,hG7b0MtEbXx5QzbzE6C_VA
1,NZnhc2sEQy3RmzKTZnqtwQ,0,2017-01-14 21:30:33,0,GJXCdrto3ASJOqKeVWPi6Q,5.0,I *adore* Travis at the Hard Rock's new Kelly ...,0,yXQM5uF2jS6es16SJzNHfg
2,WTqjgwHlXbSFevF32_DJVw,0,2016-11-09 20:09:03,0,2TzJjDVDEuAW6MR5Vuc1ug,5.0,I have to say that this office really has it t...,3,n6-Gk65cPZL6Uz8qRm3NYw
3,ikCg8xy5JIg_NGPx-MSIDA,0,2018-01-09 20:56:38,0,yi0R0Ugj_xUx_Nek0-_Qig,5.0,Went in for a lunch. Steak sandwich was delici...,0,dacAIZ6fTM6mqwW5uxkskg
4,b1b1eb3uo-w561D0ZfCEiQ,0,2018-01-30 23:07:38,0,11a8sVPMUFtaC7_ABRkmtw,1.0,Today was my second out of three sessions I ha...,7,ssoyf2_x0EQMed6fgHeMyQ


In [18]:
from sklearn.feature_extraction import FeatureHasher

m = len(reveiw_df['business_id'].unique())
h = FeatureHasher(n_features=m, input_type='string')
f = h.transform(reveiw_df['business_id'])

In [25]:
m

4618

変換後の特徴量

In [21]:
reveiw_df['business_id'].unique()

array(['ujmEBvifdJM6h6RLv4wQIg', 'NZnhc2sEQy3RmzKTZnqtwQ',
       'WTqjgwHlXbSFevF32_DJVw', ..., 'JYzP9hYTwOQhFeCEAx2zng',
       'vdKUDXI1j89VO65Krg6VGg', 'ALUqFoEbPIXEwdX7QbyR_Q'], dtype=object)

In [22]:
f.toarray()

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [26]:
f.toarray().shape

(10000, 4618)

変換後のストレージサイズ

In [23]:
from sys import getsizeof

print('Our pandas Series, in bytes: ', getsizeof(reveiw_df['business_id']))
print('Our hashed numpy array, in bytes ', getsizeof(f))

Our pandas Series, in bytes:  790104
Our hashed numpy array, in bytes  56


変換後の解釈が難しくなるため解釈を目的とするならば適用できないが，一般的な大規模データセットを利用した機械学習においては直感的な解釈の重要性は低いため，特徴量ハッシングのメリットは大きい．

# ビンカウンティング

同じく大規模なカテゴリ変数をを扱うときに使う．  
ターゲット変数の値をそのカテゴリで集計してしまう．  
履歴データを元に統計量をとるので，リークに注意．

線形モデルのみならず，決定木モデルなどにも利用される．

In [29]:
ctr_df = pd.read_csv('data/AvazuCTRPrediction/train_subset')
ctr_df.shape

(8500, 25)

In [30]:
ctr_df.head()

,Unnamed: 0,id,click,hour,C1,banner_pos,site_id,site_domain,site_category,app_id,...,device_type,device_conn_type,C14,C15,C16,C17,C18,C19,C20,C21
0,0,1.000009e+18,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,2,15706,320,50,1722,0,35,-1,79
1,1,1.000017e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
2,2,1.000037e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15704,320,50,1722,0,35,100084,79
3,3,1.000064e+19,0,14102100,1005,0,1fbe01fe,f3845767,28905ebd,ecad2386,...,1,0,15706,320,50,1722,0,35,100084,79
4,4,1.000068e+19,0,14102100,1005,1,fe8cc448,9166c161,0569f928,ecad2386,...,1,0,18993,320,50,2161,0,35,-1,157


In [31]:
len(ctr_df['device_id'].unique())

933

カテゴリ変数であるデバイスIDは1000近くあるので，エンコーディングするのは大変．  
そこで，このカテゴリ変数ターゲット変数に対し算出した統計量を新たな特徴量とする．  
今回であればCTRがターゲットなのでクリック率などでまとめる．

In [39]:
def click_counting(x, bin_column):
    clicks = pd.Series(x[x['click']>0][bin_column].value_counts(), name='clicks')
    no_clicks = pd.Series(x[x['click']<1][bin_column].value_counts(), name='no_clicks')
    
    counts = pd.DataFrame([clicks, no_clicks]).T.fillna('0')
    counts['total_clicks'] = counts['clicks'].astype('int64') + counts['no_clicks'].astype('int64')
    return counts
    
def bin_counting(counts):
    counts['N+'] = counts['clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    counts['N-'] = counts['no_clicks'].astype('int64').divide(counts['total_clicks'].astype('int64'))
    # 簡易版オッズ比，二値変数間（クリックしたか/しなかったなど）の関係の強さを測る
    counts['log_N+'] = counts['N+'].divide(counts['N-'])
    
    # Bin Countingのプロパティを返すだけの場合，ここでフィルタリングする
    bin_counts = counts.filter(items=['N+', 'N-', 'log_N+'])
    return counts, bin_counts

device_idを対象としたビンカウンティング

In [42]:
bin_column = 'device_id'
device_clicks = click_counting(ctr_df.filter(items=[bin_column, 'click']), bin_column)
device_all, device_bin_counts = bin_counting(device_clicks)

In [43]:
device_all

,clicks,no_clicks,total_clicks,N+,N-,log_N+
a99f214a,1335,6077,7412,0.180113,0.819887,0.219681
135f7d9a,2,0,2,1.000000,0.000000,inf
9af87478,2,0,2,1.000000,0.000000,inf
e62f1261,2,2,4,0.500000,0.500000,1.000000
25635c83,2,0,2,1.000000,0.000000,inf
c357dbff,2,12,14,0.142857,0.857143,0.166667
7f06088c,1,0,1,1.000000,0.000000,inf
6866e114,1,0,1,1.000000,0.000000,inf
df373817,1,0,1,1.000000,0.000000,inf
72363abb,1,0,1,1.000000,0.000000,inf


In [44]:
device_bin_counts

,N+,N-,log_N+
a99f214a,0.180113,0.819887,0.219681
135f7d9a,1.000000,0.000000,inf
9af87478,1.000000,0.000000,inf
e62f1261,0.500000,0.500000,1.000000
25635c83,1.000000,0.000000,inf
c357dbff,0.142857,0.857143,0.166667
7f06088c,1.000000,0.000000,inf
6866e114,1.000000,0.000000,inf
df373817,1.000000,0.000000,inf
72363abb,1.000000,0.000000,inf


すべてのデバイスに対しビンカウンティングされたか確認

In [45]:
len(device_bin_counts)

933

# バックオフ

カウント数がかなり少ないカテゴリは「その他カテゴリ」のような形で一つにまとめてしまう方法．